In [ ]:
#import zipfile
#with zipfile.ZipFile("NONSTEM.zip", 'r') as zip_ref:
#    zip_ref.extractall("./")

In [2]:
!pip install youtube_transcript_api

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
import os
import csv
from bs4 import BeautifulSoup
import requests
import re
from youtube_transcript_api import YouTubeTranscriptApi
import time

In [4]:
def get_files_for_field(givenPath):
    myDict = {}
    for f in os.listdir(givenPath):
        if ".ipynb_checkpoints" not in f:
            print(f)
            curr_path = givenPath + f + "/"
            print(curr_path)
            only_files = [curr_file for curr_file in os.listdir(curr_path) if os.path.isfile(curr_path + curr_file)]
            print(only_files)
            myDict[f] = []
            myDict[f] = only_files
    return myDict

In [5]:
path1 = "./STEM/"
myDict_STEM = get_files_for_field(path1)

Biology
./STEM/Biology/
['Plant science and in biology_videos.tsv', 'Habitat ecology in biology_videos.tsv', 'Molecular methodology for biologists_videos.tsv', 'Principles of genetics_videos.tsv', 'Evolution and biodiversity_videos.tsv', 'Marine and terrestrial biology_videos.tsv', 'Biochemistry_videos.tsv', 'Environmental issues in biology_videos.tsv', 'Human pysiology_videos.tsv', 'Cell structure and function_videos.tsv']
Maths
./STEM/Maths/
['Problem solving methods in maths_videos.tsv', 'Analytical and computational foundations in maths_videos.tsv', 'Calculus_videos.tsv', 'Coordinate and vector geometry_videos.tsv', 'Algebra in maths_videos.tsv', 'Regression and anova_videos.tsv', 'Algorithms and applications_videos.tsv', 'Structured programming_videos.tsv', 'Probability_videos.tsv', 'Differential equations_videos.tsv']
Chemistry
./STEM/Chemistry/
['Chemistry for physical sciences_videos.tsv', 'Mineralogy and petrology_videos.tsv', 'The global Earth system_videos.tsv', 'Solid state

In [ ]:
#all fields
my_path = "./STEM_Info/"
for key in myDict_STEM.keys():
    print("Processing for " + key + " ...")
    for curr_file in myDict_STEM[key]:
        print("Processing for " + curr_file + " ...")
        start = time.time()
        if ".tsv" in curr_file:
            file_path = path1 + key + "/" + curr_file
            text_cont_instances = list(csv.reader(open(file_path, 'r'), delimiter='\t'))
            num_video_instances = len(text_cont_instances) #generally 201 instances - first instance contains the fields, other instances are the videos
            
            file_name = my_path + key + "/" + curr_file
            with open(file_name, mode='w') as video_file:
                #video_writer = csv.writer(video_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                field_names = ['video_ID', 'title', 'description', 'subtitles']
                writer = csv.DictWriter(video_file, delimiter = '\t', quotechar = '"', quoting=csv.QUOTE_MINIMAL, fieldnames = field_names)
                writer.writeheader()
            
                for idx in range(1, num_video_instances):
                    video_ID = text_cont_instances[idx][4]
                    #title - 7th item of each instance in the .tsv file
                    video_title = text_cont_instances[idx][7]
                    video_title = video_title.replace("&quot;", "\"")
                    video_title = video_title.replace("&#39;", "'")
                    video_title = video_title.replace("&amp;", "&")
                
                    print(video_ID, video_title)
                
                    # put in the "" the final part of the video in the address bar of the browser
                    
                    subtitles_video = ""
                    try:
                        transcript_list = YouTubeTranscriptApi.list_transcripts(video_ID)
                        transcript = transcript_list.find_transcript(['en'])
                        srt = transcript.fetch()
                        print(transcript)
                        #srt = YouTubeTranscriptApi.get_transcripts(video_ID)
                        #print("subtitles", str)
                        for i in srt:
                            subtitles_video += i["text"] + " "
                            
                        subtitles_video = subtitles_video.replace("\n", " ")
                        subtitles_video = subtitles_video.replace("\\n", " ")
                        #print(subtitles_video)
                    except Exception as inst:
                        print("Not generated!")
                        
                        try:
                            srt = transcript.translate('en').fetch()
                            for i in srt:
                                subtitles_video += i["text"] + " "
                                
                            subtitles_video = subtitles_video.replace("\n", " ")
                            subtitles_video = subtitles_video.replace("\\n", " ")
                        except Exception as inst:
                            print("No SUBTITLE!!")
                
                    myURL = "https://www.youtube.com/watch?v=" + video_ID
                    r = requests.get(myURL, timeout=5)
                    stCode = r.status_code
                
                    if stCode == 200:
                        searchPageHtml = r.text
                        startIdx = searchPageHtml.find("shortDescription")
                        endIdx = searchPageHtml.find("isCrawlable")
                    
                        desc = searchPageHtml[startIdx + 19:endIdx-3]
                        desc = desc.replace("\u0026", "&")
                        #desc = re.sub(r'^https?:\/\/.*[\r\n]*', '', desc, flags=re.MULTILINE)
                        desc = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', desc, flags=re.MULTILINE)
                        desc = re.sub('www.*', '', desc, flags=re.MULTILINE)                   
                    
                        desc = desc.replace("\n", " ")
                        desc = desc.replace("\\n", " ")
                        writer.writerow({'video_ID': video_ID, 'title': video_title, 'description': desc, 'subtitles': subtitles_video})
                        #print(desc)
                        
                end = time.time()
                print("TIME: " + str(end - start))
                print("-----")
    print("\n")
    print("\n")
    print("***********")

Processing for Biology ...
Processing for Plant science and in biology_videos.tsv ...
8-G7D_sy7qE Plant Science: An Introduction to Botany | The Great Courses
en ("English (auto-generated)")[TRANSLATABLE]
9UvlqAVCoqY Plant Cells: Crash Course Biology #6
en ("English")[TRANSLATABLE]
2BR1zdMBhY4 GCSE Science Revision Biology "Plant Tissues"
en ("English (auto-generated)")[TRANSLATABLE]
EAoeI2gXBRg GCSE Science Revision Biology "Plant Cells"
en ("English (auto-generated)")[TRANSLATABLE]
w_x-WDdQdxI Plant Nutrition | Plants | Biology | FuseSchool
en ("English (auto-generated)")[TRANSLATABLE]
jtuX7H05tmQ Xylem and Phloem - Transport in Plants | Biology | FuseSchool
en ("English")[TRANSLATABLE]
cRCck4niz5o Plant Classification | Evolution | Biology | FuseSchool
en ("English (auto-generated)")[TRANSLATABLE]
X4L3r_XJW0I Plants
en ("English - bozemanscience")[TRANSLATABLE]
83JKzYIWKZA Plant Science Research at the University of Leeds | Faculty of Biological Sciences
en ("English (auto-generated